Importo las librerías que voy a utilizar

In [90]:
import pandas as pd
import numpy as np
import fastparquet
import pyarrow
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pickle

Traigo el dataset "train.parquet" utilizando las librerías pyarrow y fastparquet

In [91]:
df_train = pd.read_parquet('train.parquet') # Lee el dataset
df_train.head() # Imprime las 5 primeras filas del dataset

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,...,0,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn


Hago una visualización de los datos para tener una mejor comprensión del dataset. 

In [92]:
df_train.info() # Despliega todas las columnas y sus respectivos tipos de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346479 entries, 0 to 346478
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       346479 non-null  int64  
 1   url                      346479 non-null  object 
 2   region                   346479 non-null  object 
 3   region_url               346479 non-null  object 
 4   price                    346479 non-null  int64  
 5   type                     346479 non-null  object 
 6   sqfeet                   346479 non-null  int64  
 7   beds                     346479 non-null  int64  
 8   baths                    346479 non-null  float64
 9   cats_allowed             346479 non-null  int64  
 10  dogs_allowed             346479 non-null  int64  
 11  smoking_allowed          346479 non-null  int64  
 12  wheelchair_access        346479 non-null  int64  
 13  electric_vehicle_charge  346479 non-null  int64  
 14  come

In [93]:
df_train.isnull().sum() # Imprime la suma de los valores nulos de cada columna

id                              0
url                             0
region                          0
region_url                      0
price                           0
type                            0
sqfeet                          0
beds                            0
baths                           0
cats_allowed                    0
dogs_allowed                    0
smoking_allowed                 0
wheelchair_access               0
electric_vehicle_charge         0
comes_furnished                 0
laundry_options             71171
parking_options            126682
image_url                       0
description                     2
lat                          1722
long                         1722
state                           0
dtype: int64

Creo la columna 'category_price', que va a almacenar los valores categóricos de la columna 'price', divididos en bajos, medios y altos para posteriormente poder entrenar el modelo.

In [94]:
def categorise(row):  # Le asigno valor 1 a todos los valores de la columna 'price' que estén entre 0 y 999 dólares, y el valor 0 a los que superen dicho monto.
    if row['price'] >= 0 and row['price'] <= 999:
        return 1
    else: return 0
df_train['category_price'] = df_train.apply(lambda row: categorise(row), axis=1) # La función lambda agrega las categorías creadas por la función "categorize" a la nueva columna.

Muestro el nuevo dataset con las columnas dummies y la nueva columna categórica.

In [95]:
df_train.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state,category_price
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id,0
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co,0
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,...,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va,0
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,...,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az,0
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn,1


Elimino algunas columnas.

In [96]:
df_train.drop(['id', 'url', 'region_url', 'image_url', 'description', 'lat', 'long','laundry_options', 'parking_options', 'region', 'type', 'state', 'price'],axis=1, inplace=True) 

In [97]:
df_train.head()

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,category_price
0,1200,2,2.0,1,1,1,0,0,0,0
1,694,1,1.0,1,1,1,0,0,0,0
2,900,2,2.0,0,0,1,0,0,0,0
3,1469,3,2.0,1,1,1,0,0,0,0
4,700,1,1.0,1,1,1,0,0,0,1


Separo el dataframe para poder entrenarlo

In [98]:
X = df_train.drop(columns=['category_price'])
y = df_train['category_price']

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

Entreno el modelo de cladificación

In [100]:
classifier = RandomForestClassifier(n_estimators = 100)
classifier.fit(X_train, y_train)

RandomForestClassifier()

Guardo el modelo en un archivo pkl para poder utilizarlo posteriormente.

In [101]:
pickle.dump(classifier, open('classifier.pkl', 'wb'))

In [102]:
pickled_classifier = pickle.load(open('classifier.pkl', 'rb'))

Uso el archivo pickle para hacer la predicción sobre el archivo de entrenamiento

In [103]:
y_pred = pickled_classifier.predict(X_test)

Imprimo los resultados.

In [104]:
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[43054 12815]
 [10722 37353]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.77      0.79     55869
           1       0.74      0.78      0.76     48075

    accuracy                           0.77    103944
   macro avg       0.77      0.77      0.77    103944
weighted avg       0.77      0.77      0.77    103944

Accuracy: 0.773560763488032


Traigo el dataset "test.parquet"

In [105]:
df_test = pd.read_parquet('test.parquet')

Le quito algunas columnas para que quede con las mismas variables que el archivo de entrenamiento

In [106]:
df_test.drop(['id', 'url', 'region_url', 'image_url', 'description', 'lat', 'long','laundry_options', 'parking_options', 'region', 'type', 'state'],axis=1, inplace=True)

Uso el archivo pickle para hacer la predicción sobre el archivo de testeo.

In [107]:

pred = pickled_classifier.predict(df_test)
pred

array([1, 0, 0, ..., 0, 0, 1])

In [108]:
len(pred)

38498

Creo el archivo csv que contiene las predicciones.

In [109]:
MonicaV84 = pd.DataFrame(data=pred, columns=['pred']).set_index('pred')
MonicaV84

""
pred
1
0
0
0
0
...
0
0
0


In [110]:
MonicaV84.to_csv('MonicaV84.csv')